# Pandas accessor for `redflag`

In [2]:
import pandas as pd

df = pd.read_csv("https://geocomp.s3.amazonaws.com/data/RPC_simple.csv")

df.head()

,Vp,Vs,rho,Lithology
0,3045.6,1595.7,2.109121,sandstone
1,3000.6,1517.1,2.090342,sandstone
2,3363.6,2041.5,2.131990,sandstone
3,3195.3,1606.2,2.184939,sandstone
4,4237.5,2448.6,2.472231,sandstone


In [3]:
import redflag as rf

rf.imbalance_degree(df['Lithology'])

-1.0

In [1]:
@pd.api.extensions.register_dataframe_accessor("redflag")
class RedflagAccessor:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj

    def imbalance_degree(self, target=None):
        return rf.imbalance_degree(self._obj[target])


In [5]:
df.redflag.imbalance_degree(target='Lithology')

-1.0

Noice.